In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import logging
from pandas import DataFrame
import seaborn as sns
import numpy as np

In [ ]:
filepath = Path.cwd().parent / "data" / "raw" / "mental-heath-in-tech-2016_20161114.csv"

In [ ]:
def setup_logger(name: str, log_file: Path = None, level=logging.INFO):
    """Logger mit Konsolen- und optionaler Dateiausgabe."""
    logger = logging.getLogger(name)
    logger.setLevel(level)


    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    # Konsolen-Handler
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

    # Datei-Handler (falls gewünscht)
    if log_file:
        file_handler = logging.FileHandler(log_file)
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)

    return logger



In [ ]:
logger = setup_logger(__name__, log_file="exp.log")

In [ ]:
def robust_csv_read(filepath: Path) -> DataFrame:
    try:
        if not Path(filepath).is_file():
            raise FileNotFoundError(f"Datei ({filepath}) not found")

        df = pd.read_csv(
            filepath_or_buffer=filepath,
            sep=",",
            engine="python",
            on_bad_lines="warn"
        ).convert_dtypes(dtype_backend="numpy_nullable")

        logger.info(f"Erfolgreich Eingelesen: {str(Path)[-20:]} \n(Zeilen: {len(df)})")
        return df

    except pd.errors.EmptyDataError:
        logger.warning(f"Leere oder korrupte Datei: {filepath}")
        return pd.DataFrame()

    except pd.errors.ParserError as e:
        logger.error(f"CSV-Parsing-Fehler in {filepath}: {str(e)}", exc_info=True)
        return pd.DataFrame() # Fallback

    except Exception as e:
        logger.critical(
            f"Unbekannter Fehler beim Lesen von {filepath}: {str(e)}",
            exc_info=True
        )
        raise

    finally:
        logger.debug(f"CSV-Lesevorgang abgeschlossen für: {filepath}")

In [ ]:
df = robust_csv_read(filepath)

In [ ]:
missing_percentage = df.isna().mean() * 100

In [ ]:
missing_percentage = missing_percentage.to_frame()

In [ ]:
missing_percentage

In [ ]:
missing_percentage = missing_percentage.reset_index()
missing_percentage.columns = ["Questions", "isna%"]

In [ ]:
rows_more_than_30_percent_isna = missing_percentage[missing_percentage['isna%'] > 20]

In [ ]:
x = rows_more_than_30_percent_isna["Questions"].str[:20].apply(lambda x: x + "...")
y = rows_more_than_30_percent_isna["isna%"].to_list()
sns.set_theme(style="whitegrid", context="paper")  # Stil optimiert für Papers

plt.figure(dpi=300)

# Barplot mit expliziten Positionen
x_pos = np.arange(len(x))
plt.bar(x=x_pos, height=y, width=0.6)  # width für schmalere Balken

# X-Achse mit korrekter Zuordnung
plt.xticks(ticks=x_pos, labels=x, rotation=65, ha='right')  #

# Titel und Achsen
plt.title("Fragen mit mehr als 20% fehlender Werte", pad=20, fontsize=10)
plt.xlabel("Fragen aus dem Fragebogen", fontsize=10)
plt.ylabel("% fehlende Werte", fontsize=10)
plt.axhline(y=25, color="red", linestyle="--", linewidth=1.5, label="Grenzwert (25%)")


plt.tight_layout()
plt.legend()
plt.grid(False)
# plt.show()
plt.savefig(Path.cwd().parent / "Plots" / "Fragen_mit_mehr_als_30%_fehlender_Werte.png", dpi=300)


In [ ]:
df.head()

In [ ]:
questions = rows_more_than_30_percent_isna["Questions"]
percentages = rows_more_than_30_percent_isna["isna%"]

table_df = pd.DataFrame({
    "Fragen": questions,
    "Fehlende Werte (%)": percentages
}).convert_dtypes(dtype_backend="numpy_nullable")

table_df["Fragen"] = table_df["Fragen"]
table_df["Fehlende Werte (%)"] = table_df["Fehlende Werte (%)"].round(2).astype(pd.Float32Dtype())

In [ ]:
html_table = table_df.to_html(
    index=False,
    classes="table table-striped",

)
path = Path.cwd().parent / "Plots" / "Fragen_mit_mehr_als_20_%_fehlender_Werte.html"
with open(path, "w", encoding="utf-8") as f:
    f.write(html_table)

In [ ]:
# 1. Entfernung aller Fragen, die mehr als 25% fehlende Werte haben
threshold = 0.25
cols_to_drop = df.columns[df.isna().mean() > threshold].to_list()
df_temp_1 = df.drop(columns=cols_to_drop)
logger.info(f"Spalten gelöscht, welche mehr als 25% fehlende Werte haben: \n{len(cols_to_drop)} Spalten gelöscht\n"
            f"{df.shape[1] - len(cols_to_drop)} Spalten übrig")

In [ ]:
# Jede Spalte isoliert betrachtet bereinigen


In [133]:
# 1. Spalte
unique_values = df.iloc[:, 0].value_counts(dropna=False)
unique_values

Are you self-employed?
0    1146
1     287
Name: count, dtype: Int64

In [134]:
# 2. Spalte
unique_values = df_temp_1.iloc[:, 1].value_counts(dropna=False)
unique_values

How many employees does your company or organization have?
26-100            292
<NA>              287
More than 1000    256
100-500           248
6-25              210
500-1000           80
1-5                60
Name: count, dtype: Int64

In [137]:
df_columns = df.columns.to_list()
df_unique_values = df.nunique(dropna=False).to_list()
df_isna = df.isna().any().replace({True: "JA", False: "NEIN"}).to_list()

In [140]:
df_html = pd.DataFrame({
    "Frage": df_columns,
    "N_Unique": df_unique_values,
    "IsNa": df_isna,
}).to_html(index=False)

In [141]:
path2 = Path.cwd().parent / "Plots" / "Übersicht_AnfangsDF.html"
with open(path2, "w", encoding="utf-8") as f:
    f.write(df_html)

In [169]:
df_temp_1.iloc[:, 1].value_counts(dropna=False)

How many employees does your company or organization have?
26-100            292
<NA>              287
More than 1000    256
100-500           248
6-25              210
500-1000           80
1-5                60
Name: count, dtype: Int64

In [172]:
col = "How many employees does your company or organization have?"
threshold = int(df_temp_1.shape[1]*0.75)
n2 = df_temp_1.shape[0] - df_temp_1[df_temp_1[col].isna()].dropna(thresh=threshold).shape[0]

print(f"{n2} Zeilen gelöscht, die in {col} isna haben")

-1146 Zeilen gelöscht


In [176]:
threshold = int(df_temp_1.shape[1] * 0.75)

# Zeilen behalten, die entweder
# (a) keinen NaN in col haben  ODER
# (b) genug gültige Werte besitzen
df_temp_2 = df_temp_1[
    (~df_temp_1[col].isna()) &
    (df_temp_1.notna().sum(axis=1) >= threshold)
]

n2 = df_temp_1.shape[0] - df_temp_2.shape[0]
print(f"{n2} Zeilen gelöscht, die in '{col}' NaN sind und < {threshold} gültige Werte haben.")


315 Zeilen gelöscht, die in 'How many employees does your company or organization have?' NaN sind und < 35 gültige Werte haben.


In [200]:
threshold = int(df_temp_1.shape[1]*0.75)
df_temp_2 = df_temp_1.dropna(thresh=threshold)
print(f"{df_temp_1.shape[0] - df_temp_2.shape[0]} Zeilen gelöscht, die mehr als {df_temp_1.shape[1] - threshold} leere Spalten haben")

315 Zeilen gelöscht, die mehr als 12 leere Spalten haben


In [201]:
df_temp_2.iloc[:, 1].value_counts(dropna=False)

How many employees does your company or organization have?
26-100            284
More than 1000    253
100-500           243
6-25              203
500-1000           77
1-5                58
Name: count, dtype: Int64

In [202]:
# Ordinal Encoding
size_order = ["1-5", "6-25", "26-100", "100-500", "500-1000", "More than 1000"]

from sklearn.preprocessing import OrdinalEncoder
ordEnc = OrdinalEncoder(categories=[size_order])
df_temp_2.loc[:, col] = ordEnc.fit_transform(df_temp_2[[col]]).ravel()

C:\Users\achim\AppData\Local\Temp\ipykernel_27596\3370576033.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2. 1. 1. ... 4. 3. 3.]' has dtype incompatible with string, please explicitly cast to a compatible dtype first.
  df_temp_2.loc[:, col] = ordEnc.fit_transform(df_temp_2[[col]]).ravel()


In [216]:
df_temp_2[col3].value_counts(dropna=False)

Does your employer provide mental health benefits as part of healthcare coverage?
Yes                                521
I don't know                       311
No                                 208
Not eligible for coverage / N/A     78
Name: count, dtype: Int64

In [211]:
col3 = df_temp_2.columns[3]

In [227]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, drop="first")
encoded_data = ohe.fit_transform(df_temp_2[[col3]])

categories = ohe.categories_[0][1:]


In [228]:
new_columns = [f"{col3}_{category}" for category in categories]

encoded_df = pd.DataFrame(encoded_data, columns=new_columns,
                          index=df_temp_2.index)

df_temp_2 = df_temp_2.drop(col3, axis=1)
df_temp_2 = pd.concat([df_temp_2, encoded_df], axis=1)

In [236]:
df_temp_2.to_pickle("df_temp_2.pkl")

In [238]:
df_temp_2.iloc[:, 4].value_counts(dropna=False)

Does your employer offer resources to learn more about mental health concerns and options for seeking help?
No              511
I don't know    316
Yes             291
Name: count, dtype: Int64

In [239]:
ohe = OneHotEncoder(sparse_output=False, drop="first")

col4 = df_temp_2.columns[4]
encoded_data1 = ohe.fit_transform(df_temp_2[[col4]])

categories = ohe.categories_[0][1:]

In [240]:
new_cols = [f"{col4}_{category}" for category in categories]
encoded_df1 = pd.DataFrame(
    encoded_data1,
    columns=new_cols,
    index=df_temp_2.index
)

df_temp_2 = df_temp_2.drop(columns=col4, axis=1)
df_temp_2 = pd.concat([df_temp_2, encoded_df1], axis=1)

In [249]:
df_temp_2.iloc[:, 5].value_counts(dropna=False)

If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:
Somewhat easy                 272
Very easy                     213
Somewhat difficult            193
Neither easy nor difficult    178
I don't know                  148
Very difficult                114
Name: count, dtype: Int64

In [251]:
df_temp_2[df_temp_2.columns[5]] = df_temp_2[df_temp_2.columns[5]].replace({"I don't know": "Neither easy nor difficult"})

In [273]:
cat_order = ["Very difficult", "Somewhat difficult", "Neither easy nor difficult", "Somewhat easy", "Very easy"]

oe = OrdinalEncoder(categories=[cat_order])
df_temp_2.iloc[:, 5] = oe.fit_transform(df_temp_2[[df_temp_2.columns[5]]]).ravel()


C:\Users\achim\AppData\Local\Temp\ipykernel_27596\3003350805.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4. 3. 2. ... 3. 1. 1.]' has dtype incompatible with string, please explicitly cast to a compatible dtype first.
  df_temp_2.iloc[:, 5] = oe.fit_transform(df_temp_2[[df_temp_2.columns[5]]]).ravel()


In [266]:
df_temp_2.loc[:, 5]

0       4.0
1       3.0
2       2.0
4       2.0
5       3.0
       ... 
1425    3.0
1426    3.0
1427    3.0
1430    1.0
1431    1.0
Name: 5, Length: 1118, dtype: float64

In [278]:
df_temp_2["If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:"].value_counts()

If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:
2.0    326
3.0    272
4.0    213
1.0    193
0.0    114
Name: count, dtype: int64

In [276]:
df_temp_2 = df_temp_2.drop(5, axis=1)

In [277]:
df_temp_2.columns

Index(['Are you self-employed?',
       'How many employees does your company or organization have?',
       'Is your employer primarily a tech company/organization?',
       'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
       'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
       'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:',
       'Do you think that discussing a mental health disorder with your employer would have negative consequences?',
       'Do you think that discussing a physical health issue with your employer would have negative consequences?',
       'Would you feel comfortable discussing a mental health disorder with your coworkers?',
       'Would you feel comfortable discussing a mental health disorder with your direct s

In [280]:
df_temp_2.iloc[:, 6].value_counts(dropna=False)

Do you think that discussing a mental health disorder with your employer would have negative consequences?
Maybe    475
No       428
Yes      215
Name: count, dtype: Int64

In [281]:
col_6_order = ["Yes", "Maybe", "No"]

oe6 = OrdinalEncoder(categories=[col_6_order])
df_temp_2.iloc[:, 6] = oe6.fit_transform(df_temp_2[[df_temp_2.columns[6]]]).ravel()



C:\Users\achim\AppData\Local\Temp\ipykernel_27596\493931991.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2. 2. 1. ... 2. 1. 1.]' has dtype incompatible with string, please explicitly cast to a compatible dtype first.
  df_temp_2.iloc[:, 6] = oe6.fit_transform(df_temp_2[[df_temp_2.columns[6]]]).ravel()


In [282]:
df_temp_2.iloc[:, 6].value_counts(dropna=False)

Do you think that discussing a mental health disorder with your employer would have negative consequences?
1.0    475
2.0    428
0.0    215
Name: count, dtype: int64

In [283]:
col_7 = df_temp_2.columns[7]

In [290]:
df_temp_2[col_7].value_counts(dropna=False)

Do you think that discussing a physical health issue with your employer would have negative consequences?
2.0    816
1.0    263
0.0     39
Name: count, dtype: int64

In [289]:
col_7_order = ["Yes", "Maybe", "No"]

oe_7 = OrdinalEncoder(categories=[col_7_order])
df_temp_2.loc[:, col_7] = oe_7.fit_transform(df_temp_2[[col_7]]).ravel()

ValueError: could not convert string to float: 'Yes'

In [294]:
df_temp_2[col_7].nunique()

3

In [296]:
df_temp_2.iloc[:, 8].value_counts(dropna=False)

Would you feel comfortable discussing a mental health disorder with your coworkers?
Maybe    467
No       382
Yes      269
Name: count, dtype: Int64

In [297]:
col_8 = df_temp_2.columns[8]

In [300]:
ohe_8 = OneHotEncoder(sparse_output=False, drop="first")

encoded_data8 = ohe_8.fit_transform(df_temp_2[[col_8]])

categories = ohe_8.categories_[0][1:]
new_cols_8 = [f"{col_8}_{cat}" for cat in categories]

encoded_df8 = pd.DataFrame(encoded_data8, columns=new_cols_8, index=df_temp_2.index)

df_temp_2 = df_temp_2.drop(columns=[col_8])
df_temp_2 = pd.concat([encoded_df8, df_temp_2], axis=1)

In [320]:
df_temp_2.head(10)

,Would you feel comfortable discussing a mental health disorder with your coworkers?_No,Would you feel comfortable discussing a mental health disorder with your coworkers?_Yes,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,...,What is your gender?,What country do you live in?,What country do you work in?,Which of the following best describes your work position?,Do you work remotely?,Does your employer provide mental health benefits as part of healthcare coverage?_No,Does your employer provide mental health benefits as part of healthcare coverage?_Not eligible for coverage / N/A,Does your employer provide mental health benefits as part of healthcare coverage?_Yes,Does your employer offer resources to learn more about mental health concerns and options for seeking help?_No,Does your employer offer resources to learn more about mental health concerns and options for seeking help?_Yes
0,0.0,0.0,0,2.0,1,No,I don't know,4.0,2.0,2.0,...,Male,United Kingdom,United Kingdom,Back-end Developer,Sometimes,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0,1.0,1,Yes,Yes,3.0,2.0,2.0,...,male,United States of America,United States of America,Back-end Developer|Front-end Developer,Never,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0,1.0,1,No,I don't know,2.0,1.0,2.0,...,Male,United Kingdom,United Kingdom,Back-end Developer,Always,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0,1.0,0,No,No,2.0,0.0,1.0,...,Female,United States of America,United States of America,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes,0.0,0.0,1.0,1.0,0.0
5,0.0,0.0,0,5.0,1,No,Yes,3.0,0.0,0.0,...,Male,United Kingdom,United Kingdom,DevOps/SysAdmin|Support|Back-end Developer|Fro...,Sometimes,0.0,0.0,1.0,0.0,1.0
6,0.0,0.0,0,2.0,1,No,I don't know,3.0,2.0,2.0,...,M,United States of America,United States of America,Back-end Developer,Sometimes,0.0,0.0,0.0,1.0,0.0
7,0.0,0.0,0,5.0,1,No,Yes,4.0,2.0,2.0,...,female,United States of America,United States of America,Dev Evangelist/Advocate|Back-end Developer,Always,0.0,0.0,1.0,0.0,1.0
8,0.0,1.0,0,2.0,0,No,I don't know,0.0,0.0,0.0,...,Female,United States of America,United States of America,Support|Back-end Developer|One-person shop,Sometimes,0.0,0.0,0.0,1.0,0.0
10,0.0,0.0,0,2.0,1,Yes,Yes,4.0,2.0,2.0,...,Male,United States of America,United States of America,Front-end Developer,Never,0.0,0.0,1.0,0.0,1.0
11,1.0,0.0,0,3.0,0,No,I don't know,1.0,0.0,2.0,...,Male,United States of America,United States of America,Executive Leadership,Sometimes,0.0,0.0,1.0,0.0,0.0


In [322]:
df_temp_2.to_pickle("df_temp_2.pkl")